# COMMOT (Modified) — Toy Demo with Modulators (Jupyter)

這份 Notebook 會：
1. 準備一個極小的 AnnData（6 個 cell × 4 個基因）。
2. 定義兩組配受體（其中一組為**異質複合體**）。
3. 建立四類調節向量（AG/AN/CS/CI），對異質複合體自動以 `min/ave` 聚合。
4. 呼叫我們改造後的 `spatial_communication`（含 $K' = M\,e^{-C/\varepsilon}$ 與修改版 Sinkhorn）。
5. 視覺化：耦合矩陣熱圖、空間散點、sender/receiver 彙總。

In [ ]:
# Imports
import sys, os
import numpy as np
import pandas as pd
import anndata
import matplotlib.pyplot as plt

# 讓本地的 commot_mod/ 可被匯入（請把本 Notebook 放在專案根目錄）
sys.path.insert(0, os.getcwd())
from commot_mod import spatial_communication
print("Python:", sys.executable)

## 1) 準備一筆小資料

In [ ]:
genes = ["LigA", "RecA", "Tgfbr1", "Tgfbr2"]
X = np.array([
    [3.0, 0.2, 0.1, 0.1],
    [2.5, 0.1, 0.1, 0.1],
    [0.2, 2.8, 0.3, 0.2],
    [0.2, 2.2, 0.2, 0.2],
    [0.2, 0.3, 2.5, 2.6],
    [0.2, 0.3, 2.2, 2.4],
], float)
adata = anndata.AnnData(X=X, var=pd.DataFrame(index=genes))
adata.obsm["spatial"] = np.array([
    [0.0, 0.0],
    [0.6, 0.1],
    [1.2, 0.0],
    [1.8, 0.1],
    [1.1, 0.9],
    [1.7, 1.0],
], float)
adata

## 2) 定義配受體（含異質複合體）

In [ ]:
df_ligrec = pd.DataFrame([
    ["LigA", "RecA", "PathwayX"],
    ["LigA", "Tgfbr1_Tgfbr2", "PathwayX"],
], columns=["ligand","receptor","pathway"])
df_ligrec

## 3) 四類調節向量（AG/AN/CS/CI）

In [ ]:
AG_LigA = np.array([0.5,0.6,0.1,0.1,0.1,0.1])
AN_LigA = np.array([0.0,0.0,0.2,0.2,0.2,0.2])
CS_RecA = np.array([0.1,0.1,0.7,0.7,0.1,0.1])
CI_RecA = np.array([0.2,0.2,0.0,0.0,0.2,0.2])
CS_Tgfbr1 = np.array([0.1,0.1,0.2,0.3,0.8,0.7])
CS_Tgfbr2 = np.array([0.1,0.1,0.2,0.3,0.7,0.8])
CI_Tgfbr1 = np.array([0.2,0.2,0.3,0.3,0.0,0.1])
CI_Tgfbr2 = np.array([0.2,0.2,0.3,0.3,0.1,0.0])

modulators = {
    "AG": {"LigA": AG_LigA},
    "AN": {"LigA": AN_LigA},
    "CS": {"RecA": CS_RecA, "Tgfbr1": CS_Tgfbr1, "Tgfbr2": CS_Tgfbr2},
    "CI": {"RecA": CI_RecA, "Tgfbr1": CI_Tgfbr1, "Tgfbr2": CI_Tgfbr2},
}
modulators

## 4) 執行改造後的 `spatial_communication`

In [ ]:
spatial_communication(
    adata,
    database_name="toyDB",
    df_ligrec=df_ligrec,
    pathway_sum=True,
    heteromeric=True,
    heteromeric_rule="min",   # 可改成 "ave" 比較
    dis_thr=1.2,
    cost_type="euc",
    cot_eps_p=1e-1,
    cot_rho=1e1,
    modulators=modulators,
    hill_Kh=0.5,
)
list(adata.obsp.keys())

## 5) 視覺化
不使用 seaborn；每張圖單獨一個 figure；不指定任何顏色。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def show_heatmap(key):
    mat = adata.obsp[key]
    M = mat.toarray() if hasattr(mat, "toarray") else np.asarray(mat)
    plt.figure()
    plt.imshow(M)
    plt.title(key)
    plt.colorbar()
    plt.xlabel("receiver cell index")
    plt.ylabel("sender cell index")
    plt.show()

keys_to_plot = [k for k in adata.obsp.keys() if k.startswith("commot-toyDB-")]
for k in sorted(keys_to_plot):
    show_heatmap(k)

In [ ]:
# 空間散點：顯示 LigA 和 RecA 的表達大小（用點大小表現）
plt.figure()
xy = adata.obsm["spatial"]
LigA = adata[:, "LigA"].X.A1 if hasattr(adata[:, "LigA"].X, 'A1') else adata[:, "LigA"].X.ravel()
RecA = adata[:, "RecA"].X.A1 if hasattr(adata[:, "RecA"].X, 'A1') else adata[:, "RecA"].X.ravel()
plt.scatter(xy[:,0], xy[:,1], s=20 + 80*LigA, label="LigA")
plt.scatter(xy[:,0], xy[:,1], s=20 + 80*RecA, label="RecA", marker='^')
plt.legend(); plt.title("Spatial points: size ~ expression (LigA/RecA)")
plt.xlabel("x"); plt.ylabel("y"); plt.show()

In [ ]:
# Sender / Receiver 彙總長條圖
sender = adata.obsm["commot-toyDB-sum-sender"][['s-total-total']]
receiver = adata.obsm["commot-toyDB-sum-receiver"][['r-total-total']]

plt.figure(); sender.plot(kind='bar', legend=False)
plt.title("Total Sent per Cell"); plt.xlabel("cell"); plt.ylabel("sum sent")
plt.show()

plt.figure(); receiver.plot(kind='bar', legend=False)
plt.title("Total Received per Cell"); plt.xlabel("cell"); plt.ylabel("sum received")
plt.show()